<a href="https://colab.research.google.com/github/liuwang2000/colabScript/blob/main/notebooks/Pro_Prime_690M_Tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TM prediction with Prime

This tutorial demonstrates how to predict the OGT of a protein using a pretrained model from the Prime model.

We provide:

- The sequences, a FASTA file.

Goals
Obtain an predicted TM for each sequence.


## Import the necessary libraries and modules.

In [1]:
!pip install biopython==1.81

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.1 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
import os # Import the os modul
from tqdm.notebook import tqdm
from transformers import AutoModel

## Prepare data path

In [3]:
sequence_file = "/content/test.fasta"

## Load model and Tokenizer

In [4]:
!huggingface-cli login # 确保登录成功

model_path = "AI4Protein/Prime_690M"  # 再次确认模型路径
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Remove `from_flax=True` as the model might not be in Flax format
model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

model.eval()
model = model.to(device)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 10, in <module>
  File "/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "/usr/local/lib/python3.11/dist-packages/h

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

configuration_proprime.py:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- configuration_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenization_proprime.py:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- tokenization_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

## Prediction

In [ ]:
import pandas as pd
from Bio import SeqIO
from IPython.display import FileLink

tm = []
sequence_names = []  # 用于存储序列名称的列表

with torch.no_grad():
    for record in tqdm(list(SeqIO.parse(sequence_file, "fasta"))):
        sequence = str(record.seq)
        sequence_names.append(record.id)  # 获取序列名称并添加到列表
        tokenied_results = tokenizer(sequence, return_tensors="pt")
        input_ids = tokenied_results.input_ids.to(device)
        attention_mask = tokenied_results.attention_mask.to(device)
        logits = model(input_ids=input_ids, attention_mask=attention_mask).predicted_values
        tm.append(logits.item())

# 创建 pandas DataFrame
df = pd.DataFrame({'sequence_names': sequence_names, 'Prime_690M_Tm': tm})

# 将 DataFrame 保存为 CSV 文件
csv_file_path = 'tm_predictions.csv'  # 指定文件名
df.to_csv(csv_file_path, index=False)  # 保存为 CSV 文件，不包含索引
# 指定保存路径
save_path = '/content/tm_predictions.csv'  # 修改保存路径

# 将 DataFrame 保存为 CSV 文件
df.to_csv(save_path, index=False)  # 保存到指定路径

print(f"文件已保存到: {save_path}")  # 打印保存路径信息


  0%|          | 0/17 [00:00<?, ?it/s]